In [1]:
from bs4 import BeautifulSoup
import pandas as pd 
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
#df = pd.DataFrame(columns=['name', 'color', 'size', 'price_sale', 'price'])
url_unique ='https://www.superga.com'
url_base = 'https://www.superga.com/collections/'
url_page = 'womens-flip-flops'
url = url_base+url_page
header = {"User-Agent":"Chrome/39.0"}
print(url)
page = requests.get(url, headers=header)
print(page)


https://www.superga.com/collections/womens-flip-flops
<Response [200]>


In [ ]:
soup = BeautifulSoup(page.text)
infradito = soup.find_all('a', class_="grid-product__link")

print(infradito)

[<a class="grid-product__link" href="/collections/womens-flip-flops/products/slippers-woman-4121-fanrbrw-palms-white-s111flw-a0b">
<div class="grid-product__meta cb-text-left">
<div class="grid-product__title grid-product__title--heading">   4121-FANRBRW<span class="cb-italic cb-block cb-font-500">PALMS-WHITE</span></div><div class="grid-product__price">
<span class="cb-text-saving">
  
      from
    
  
    €14,00
    <span class="cb-no-margin-left cb-text-max-price cb-text-strikethrough"> €35,00</span>
</span>
<span class="cb-block cb-text-max-price cb-lineheight-1">
    Best previous price: €35,00
  </span>
</div></div>
</a>, <a class="grid-product__link" href="/collections/womens-flip-flops/products/slippers-woman-4121-fanrbrw-white-black-hawaii-s111flw-a0i">
<div class="grid-product__meta cb-text-left">
<div class="grid-product__title grid-product__title--heading">   4121-FANRBRW<span class="cb-italic cb-block cb-font-500">WHITE-BLACK HAWAII</span></div><div class="grid-product__

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode (no UI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [ ]:
rows = []
for product in infradito:
    try:
        # Get the product URL
        url_ciabatta = product.get('href')  # Get the link from ciabattenomi
        url1 = url_unique + url_ciabatta  # Construct the full URL
        driver.get(url1)  # Load the product page

        # Wait for JavaScript to render the page
        time.sleep(5)

        # Extract the rendered HTML
        html_content = driver.page_source
        soup_ciabatta = BeautifulSoup(html_content, 'html.parser')

        # Extract product details
        name_ciabatta = soup_ciabatta.find('h1', class_='h2 product-single__title')
        colore_ciabatta = soup_ciabatta.find('span', class_='cb-italic cb-block cb-font-500')
        prezzo_scontato_ciabatta = soup_ciabatta.find('span', class_='related__price mr')
        prezzo_originale_ciabatta = soup_ciabatta.find('span', class_='related__on-sale')
        colors_divs = soup_ciabatta.find_all('div', class_='related__color-name skeleton-effect')
        sizes = soup_ciabatta.find('ul', class_='related__select-optgroup-options')

        nomi = name_ciabatta.text.strip()
        colore =colore_ciabatta.text.strip()
        prezzo_scontato =prezzo_scontato_ciabatta.text.strip()
        prezzo_originale = prezzo_originale_ciabatta.text.strip()
        
        # Remove color info from the name
        if colore in nomi:
            nomi = nomi.replace(colore, "")

        all_colors = []
        
        # Get all colors
        for color_div in colors_divs:
            color_name = color_div.text.strip()
            if color_name:  # Ensure the color name is not empty
                all_colors.append(color_name)

        # Get all sizes
        all_size = []

        for size in sizes :
            size_val = size.text.strip()
            if size :
                all_size.append(size_val)
        print(nomi)
        print(colore)
        print(prezzo_scontato)
        print(prezzo_originale)
        print(all_colors)
        print(all_size)
        # Loop through all colors and sizes to add combinations to rows
        for color in all_colors:
            for size in all_size:
                rows.append({
                    'name': nomi,                 # Product name
                    'color': color,               # Color option
                    'size': size[:2],                 # Size option
                    'price_sale': prezzo_scontato[1:],  # Sale price
                    'price': prezzo_originale[1:]      # Original price
                })
        print(rows)
    except Exception as e:
        print(f"Error processing product: {product}. Error: {e}")

# Create DataFrame from the rows
df = pd.DataFrame(rows)


# Display the DataFrame
print(df)

4121-FANRBRW
PALMS-WHITE
€14.00
€35.00
['PALMS-BLACK', 'PALMS-WHITE', 'WHITE-BLACK HAWAII']
['35', '36', '37', '38', '39', '40', '41', '42']
[{'name': '4121-FANRBRW', 'color': 'PALMS-BLACK', 'size': '35', 'price_sale': '14.00', 'price': '35.00'}, {'name': '4121-FANRBRW', 'color': 'PALMS-BLACK', 'size': '36', 'price_sale': '14.00', 'price': '35.00'}, {'name': '4121-FANRBRW', 'color': 'PALMS-BLACK', 'size': '37', 'price_sale': '14.00', 'price': '35.00'}, {'name': '4121-FANRBRW', 'color': 'PALMS-BLACK', 'size': '38', 'price_sale': '14.00', 'price': '35.00'}, {'name': '4121-FANRBRW', 'color': 'PALMS-BLACK', 'size': '39', 'price_sale': '14.00', 'price': '35.00'}, {'name': '4121-FANRBRW', 'color': 'PALMS-BLACK', 'size': '40', 'price_sale': '14.00', 'price': '35.00'}, {'name': '4121-FANRBRW', 'color': 'PALMS-BLACK', 'size': '41', 'price_sale': '14.00', 'price': '35.00'}, {'name': '4121-FANRBRW', 'color': 'PALMS-BLACK', 'size': '42', 'price_sale': '14.00', 'price': '35.00'}, {'name': '4121-FAN

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('db_infradito.csv', index=False)
